# Preprocessing Bike Sharing Data

In [1]:
import numpy as np
import pandas as pd
import datetime
from sqlalchemy import create_engine
import requests
import time

/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### Datasets

- Raw data: raw.csv

- Categorized in Trip and accessible: preprocessed.csv

- Cleaned according to plausability: cleaned.csv

- routed routed.csv

- Trip Dataset: pseudonomysed.csv
    - pseudonomysed
    - trip ID, start time, end time, start location, stop location, duration, estimated distance<

# Preprocessing

## get data from database

In [ ]:
dbhost = ''
dbname = ''
dbuser = ''
dbpassword = ''

In [ ]:
sql = """
select *
from (
    select  "bikeId", "providerId", "timestamp", latitude, longitude,
			lag("bikeId") over (order by "bikeId" ASC, "timestamp" ASC) as prev_id,
            lead("bikeId") over (order by "bikeId" ASC, "timestamp" ASC) as next_id,
            lag("latitude") over (order by "bikeId" ASC, "timestamp" ASC) as prev_lat,
			lag("longitude") over (order by "bikeId" ASC, "timestamp" ASC) as prev_lon,
            lead("latitude") over (order by "bikeId" ASC, "timestamp" ASC) as next_lat,
            lead("longitude") over (order by "bikeId" ASC, "timestamp" ASC) as next_lon
    from public."bikeLocations"
    )x
	where ("bikeId" <> next_id) or
		   ("bikeId" <> prev_id) or
		   (latitude <> next_lat or longitude <> next_lon or latitude <> prev_lat or longitude <> prev_lon)
	
	--keep first and last instance of bike (bikeId <> prev_id or bikeId <> next_id)
	--keep all changes inbetween, where bike_Id is equal to next_id but lat or lon changed to prev or next
"""

engine = create_engine('postgresql://' + dbuser + ':' + dbpassword + '@' + dbhost + ':5432/' + dbname)
df = pd.read_sql_query(sql,con=engine)

In [ ]:
# df.to_csv('../../data/raw.csv', index_label= 'id')

## preprocess

In [169]:
df = pd.read_csv('../../data/raw.csv', parse_dates=['timestamp'])

In [170]:
# correct wrong lat lons 
# get mode (trip or is bike accessible)

def preprocess(df):
    
    # for a certain period the lat lon of one provider were stored the wrong way 
    # switch lat lon where it's wrong (for Berlin)
    temp = df.loc[(df.longitude > 40),'latitude'].copy(deep=True)

    df.loc[(df.longitude > 40),'latitude'] = df.loc[(df.longitude > 40),'longitude']
    df.loc[(df.longitude > 40),'longitude'] = temp
    
    temp0 = df.loc[(df.next_lon > 40),'next_lat'].copy(deep=True)

    df.loc[(df.next_lon > 40),'next_lat'] = df.loc[(df.next_lon > 40),'next_lon']
    df.loc[(df.next_lon > 40),'next_lon'] = temp0
    
    temp1 = df.loc[(df.prev_lon > 40),'prev_lat'].copy(deep=True)
    
    df.loc[(df.prev_lon > 40),'prev_lat'] = df.loc[(df.prev_lon > 40),'prev_lon']
    df.loc[(df.prev_lon > 40),'prev_lon'] = temp1
    
    MODE_TRIP = 'trip'
    MODE_ACCESS = 'accessible'

    df['end_timestamp'] = None
    df['mode'] = None

    df.sort_values(by=['bikeId', 'timestamp'], inplace = True)
    df['end_timestamp'] = df.timestamp.shift(-1)
    
    df['next_lat'] = np.where(df.bikeId == df.bikeId.shift(-1), df.latitude.shift(-1), -1)\n
    df['next_lon'] = np.where(df.bikeId == df.bikeId.shift(-1), df.longitude.shift(-1), -1)\n

    # if the bike has moved its a trip, otherwise the bike was accessible at its location
    # round to 3 digits for lat lon (trip should be longer than approx. 100 meters)
    df['mode'] = np.where((df.latitude.round(3) != df.next_lat.round(3)) | (df.longitude.round(4) != df.next_lon.round(4)), MODE_TRIP, MODE_ACCESS)
    
    # if bike Id is not the same as next, then this is the final time stamp and the next_timestamp is set to None
    df['end_timestamp'] = df.end_timestamp.where(df.bikeId == df.bikeId.shift(-1),None, axis=0)
    
    # delete last instances of all bikes
    df = df[df.next_lat.notnull()]
    
    return df

In [171]:
df = preprocess(df)

In [172]:
# df.to_csv('../../data/preprocessed.csv', index_label= 'id')

## Routing

In [4]:
def routing(start_lon, start_lat, end_lon, end_lat):
    start = str(start_lon) + ',' + str(start_lat)
    end = str(end_lon) + ',' + str(end_lat)
    params = {'start':start,
              'end':end,
              'modal':'bicycle'
             }
    r = requests.get('http://localhost:1717/route', params = params)
    response = r.json()
    return response

In [181]:
df = pd.DataFrame()
for chunk in pd.read_csv('../../data/preprocessed.csv', index_col='id', parse_dates=['timestamp', 'end_timestamp'],chunksize=100000):
    start = time.time()
    chunk['response'] = None
    chunk['geometry'] = None
    chunk['distance'] = None
    chunk['calc_time'] = None
    chunk['waypoints'] = None
    
    # only do routing for trips
    selection = (chunk['mode']=='trip') & (chunk.next_lat.notnull())
    chunk.loc[selection, 'response'] = chunk.loc[selection].apply(lambda row: routing(row.longitude, row.latitude, row.next_lon, row.next_lat), axis=1)
    chunk.loc[selection, 'geometry'] = chunk.loc[selection]['response'].apply(lambda row: row['routes'][0]['geometry'])
    chunk.loc[selection, 'distance'] = chunk.loc[selection]['response'].apply(lambda row: row['routes'][0]['distance'])
    chunk.loc[selection, 'calc_time'] = chunk.loc[selection]['response'].apply(lambda row: row['routes'][0]['duration'])
    chunk.loc[selection, 'waypoints'] = chunk.loc[selection]['response'].apply(lambda row: row['waypoints'])
    end = time.time()
    print(end - start)
    df = df.append(chunk)

214.57879996299744
223.48249912261963
214.69404888153076
189.998055934906
177.86353087425232
178.07002592086792
178.5570878982544
490.3402931690216
514.6639800071716
230.62421798706055
204.9859230518341
166.9845311641693
173.87214183807373
163.3482518196106
156.35346794128418
154.9044930934906
168.71858096122742
179.56365704536438
154.8981430530548
153.71806001663208
161.40800881385803
160.61402201652527
484.7227940559387
165.4938759803772
144.62416791915894
160.95510387420654
188.71538925170898
160.1147141456604
156.14063096046448
145.88051581382751
151.20413494110107
5570.233852863312
168.08731317520142
156.1642780303955
168.08376622200012
180.47953414916992
164.22090196609497
177.3228931427002
9975.520352125168
155.88235807418823
154.1496877670288
146.9719421863556
1111.4445898532867
161.26306581497192
165.64914107322693
157.92316102981567
171.8003692626953
163.10643601417542
150.91775488853455
175.11917686462402
0.3518679141998291


In [182]:
# df.to_csv('../../data/routed.csv')

In [115]:
# df.to_json('../../data/routed.json')

## Pseudonomysation of uncleaned data

In [5]:
df = pd.read_csv('../../data/routed.csv', parse_dates=['timestamp','end_timestamp'])

In [40]:
# pseudonomoysed
pseudon = df.copy(deep=True)
pseudon['starttime'] = pseudon.timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
pseudon['endtime'] = pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)) if isinstance(dt, pd.Timestamp) else None)
pseudon.reset_index()
pseudon.latitude = pseudon.latitude.round(3)
pseudon.longitude = pseudon.longitude.round(3)
pseudon.next_lat = pseudon.next_lat.round(3)
pseudon.next_lon = pseudon.next_lon.round(3)
pseudon = pseudon[pseudon['mode'] == 'trip'].drop(['providerId', 'bikeId', 'timestamp', 'end_timestamp', 'next_id','prev_id', 'mode', 'prev_lat', 'prev_lon', 'response', 'calc_time','geometry', 'waypoints'], axis=1).copy(deep=True)

In [41]:
pseudon.to_csv('../../data/pseudonomysed.csv', index_label='id')

## Data Cleaning

In [160]:
# df = pd.read_csv('../../data/routed.csv', parse_dates=['timestamp','end_timestamp'])

In [183]:
def cleaning(df):
    df['duration'] = df.end_timestamp - df.timestamp
    df['speed'] = np.where(df.distance.notnull(), ((df.distance/1000) / (df.duration.apply(lambda x: x.total_seconds()/3600))), None)
 
    # delete instances with unplausible locations (outside of Berlin)
    df.drop(df[df.longitude > 13.8].index, inplace=True)
    df.drop(df[df.next_lon > 13.8].index, inplace=True)

    df.drop(df[df.longitude < 13.0].index, inplace=True)
    df.drop(df[df.next_lon < 13.0].index, inplace=True)

    df.drop(df[df.latitude > 52.7].index, inplace=True)
    df.drop(df[df.next_lat > 52.7].index, inplace=True)

    df.drop(df[df.latitude < 52.3].index, inplace=True)
    df.drop(df[df.next_lat < 52.3].index, inplace=True)

    
    # Check for unplausible data
    
    # drop trips that are longer than 24 hours
    df = df.drop(df[(df['mode'] == 'trip') & (df.duration > datetime.timedelta(hours=24))].index)

    # drop trips before 1.4.2019 
    df = df.drop(df[df['timestamp'] <  pd.Timestamp(2019,4,1)].index)
    
    # drop trips that are smaller than 100 m or faster than 30 km/h
    df = df[(df['mode']=='accessible')|((df.distance > 100) & (df.speed < 30))]
    
    return df

In [184]:
df = cleaning(df)

In [ ]:
# TODO: drop all last occurencies

In [286]:
ids = df.sort_values(by=['bikeId', 'timestamp']).groupby('bikeId').last()['id.1']

In [306]:
df = df.drop(ids.tolist())

In [307]:
df.to_csv('../../data/cleaned.csv', index_label = 'id')

## Pseudonymisation of cleaned data

https://datasmart.ash.harvard.edu/news/article/civic-analytics-network-dockless-mobility-open-letter

https://data.louisvilleky.gov/dataset/dockless-vehicles

In [308]:
# pseudonomoysed cleaned
pseudon = df.copy(deep=True)
pseudon['starttime'] = pseudon.timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
pseudon['endtime'] = np.where(pseudon.end_timestamp.isnull(), None, pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15))))
# pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)) if isinstance(dt, pd.Timestamp) else None)
pseudon.reset_index()
pseudon.latitude = pseudon.latitude.round(3)
pseudon.longitude = pseudon.longitude.round(3)
pseudon.next_lat = pseudon.next_lat.round(3)
pseudon.next_lon = pseudon.next_lon.round(3)
pseudon = pseudon[pseudon['mode'] == 'trip'].drop(['providerId', 'bikeId', 'timestamp', 'end_timestamp', 'next_id','prev_id', 'mode', 'prev_lat', 'prev_lon', 'response', 'calc_time','geometry', 'waypoints'], axis=1).copy(deep=True)

In [309]:
pseudon.to_csv('../../data/pseudonomysed_clean.csv', index_label='id')

## Pseudonymisation without Mobike 

In [77]:
# pseudonomoysed cleaned without Mobike
pseudon = df[df.providerId != 2].copy(deep=True)
pseudon['starttime'] = pseudon.timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
pseudon['endtime'] = np.where(pseudon.end_timestamp.isnull(), None, pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15))))
# pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)) if isinstance(dt, pd.Timestamp) else None)
pseudon.reset_index()
pseudon.latitude = pseudon.latitude.round(3)
pseudon.longitude = pseudon.longitude.round(3)
pseudon.next_lat = pseudon.next_lat.round(3)
pseudon.next_lon = pseudon.next_lon.round(3)
pseudon = pseudon[pseudon['mode'] == 'trip'].drop(['providerId', 'bikeId', 'timestamp', 'end_timestamp', 'next_id','prev_id', 'mode', 'prev_lat', 'prev_lon', 'response', 'calc_time','geometry', 'waypoints'], axis=1).copy(deep=True)

In [78]:
pseudon.to_csv('../../data/pseudonomysed_clean_small.csv', index_label='id')